In [23]:
import os
import shutil
from tqdm import tqdm
import random

In [24]:
ACTUAL_PATH = os.getcwd()
IMAGES = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\faces\\images\\train")
LABELS = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\faces\\labels\\train")

OUTPUT_PATH = os.path.join(ACTUAL_PATH, "..\\YOLO_train\\datasets")

In [25]:
def make_dirs(train_test):
    for type in train_test:
        os.makedirs(os.path.join(ACTUAL_PATH, OUTPUT_PATH, type, "images"), exist_ok=True)
        os.makedirs(os.path.join(ACTUAL_PATH, OUTPUT_PATH, type, "labels"), exist_ok=True)

In [26]:
def split_data(images, labels, train_number, test_number):
    data = list(zip(images, labels))
    random.shuffle(data)
    train_data = data[:train_number]
    test_data = data[train_number:train_number + test_number]
    return train_data, test_data

In [27]:
def move_to_dir(data, train_or_test):

    images_path_destination = os.path.join(ACTUAL_PATH, OUTPUT_PATH, train_or_test, "images")
    labels_path_destination = os.path.join(ACTUAL_PATH, OUTPUT_PATH, train_or_test, "labels")

    data_images = [img for img, _ in data]
    data_labels = [label for _, label in data]

    for img_name in data_images:
        img_path = os.path.join(IMAGES, img_name)
        image_path_dest = os.path.join(images_path_destination, img_name)
        shutil.copy(img_path, image_path_dest)

    for label_name in data_labels:
        label_path = os.path.join(LABELS, label_name)
        label_path_dest = os.path.join(labels_path_destination, label_name)
        shutil.copy(label_path, label_path_dest)


In [28]:
train_test = ["train", "test"]
make_dirs(train_test)

In [29]:
images = os.listdir(IMAGES)
labels = os.listdir(LABELS)
train_data, test_data = split_data(images, labels, 500, 100)

In [30]:
move_to_dir(train_data, "train")
move_to_dir(test_data, "test")